In [1]:
import pandas as pd
import requests
import warnings
import time
import multiprocessing
import threading
from concurrent.futures import ThreadPoolExecutor
from ipapy import is_valid_ipa
# from google.colab import files
warnings.filterwarnings('ignore', message='Unverified HTTPS request')

In [3]:
df = pd.read_csv("ebanglalibrary_100k_with_ipa0_106500.csv")
df_words=pd.read_csv("ipa_validation_batch1.xlsx - Sheet1.csv")


In [14]:
df_words

,ɪpas_old,Unnamed: 1,words,word_fix,ɪpas,ipa_alternate,issues,sentence,assigned person
0,ɐtlɐntɪk,0,আটলান্টিক,NaN,ɐtlɐntɪk,NaN,NaN,আটলান্টিক ও প্রশান্ত মহাসাগরকে সংযুক্ত...,Dawood
1,go,0,গ,NaN,gɔ,NaN,NaN,গ) ট্রেনটি কতক্ষণ ভ্রমণ করল?সমাধানক) আ...,NaN
2,net̪ɾokonɐ,0,নেত্রকোনা,NaN,net̪ɾokonɐ,NaN,NaN,\tনেত্রকোনা প্রতিনিধিপলাশে যুবলীগের বর্ধিত সভা...,NaN
3,mɐnuʃeɾ,0,মানুষের,NaN,mɐnuʃeɾ,NaN,NaN,\tমানুষের চোখের রং নিয়ন্ত্রণ করে কোনটি?√ক) ডি....,NaN
4,NaN,0,,NaN,NaN,NaN,NaN,প্রতি আসনের বিপরীতে প্রায় ২৪ জন করে লড়বেন।,NaN
...,...,...,...,...,...,...,...,...,...
43346,ɪ-,7,ী-,NaN,ɪ-,NaN,NaN,"কার চিহ্ন দিয়ে শব্দ তৈরি কর :-ি, ী-, -ৈ, ে া-,...",NaN
43347,ɪʲo,6,ীয়,NaN,ɪʲo,NaN,NaN,অনুষ্ঠানের উদ্ধোধক ছিলেন জাতীয় পার্টির কেন্দ ী...,NaN
43348,e,9,ে,NaN,e,NaN,NaN,"কার চিহ্ন দিয়ে শব্দ তৈরি কর :-ি, ী-, -ৈ, ে া-,...",NaN
43349,-,12,ৗ-,NaN,-,NaN,NaN,"কার চিহ্ন দিয়ে শব্দ তৈরি কর :-ি, ী-, -ৈ, ে া-,...",NaN


In [27]:
def replace_func(subset):
  for row in subset.itertuples():
    ipa=row.ipa+"%"
    for i, ch in enumerate(ipa):
      if ch=='ʲ' and is_valid_ipa(ipa[i+1])==False:
        ipa = ipa[:i] + "e̯" + ipa[i+1:]
        print(f"{row.Index}:{i}{ipa[i+1]} {ipa}")
    ipa=ipa.replace("a","ɐ")
    ipa=ipa.replace("i","ɪ")
    ipa=ipa.replace("æ","ɛ")
    ipa=ipa.replace("r","ɾ")
    ipa=ipa[:-1]
    df.at[row.Index,"validated_ipa"]=ipa

    

    
    # words=row.sentence.split(" ")
    # for word in words:
    #   if word[-1]=='ʲ':
    #     word=word[:-1] + "e̯"
        
    #   for ch in word:
        

In [6]:
prefix_sentence="__$$__"
prefix_words="__##__"

In [66]:
def replace_words(subset):
  all_sentences=""
  for row in subset.itertuples():
    all_sentences+=prefix_sentence+row.sentence
  for row in df_words.itertuples():
    if pd.isna(row.ɪpas) or pd.isna(row.words):
      continue
    all_sentences=all_sentences.replace(" "+row.words+" "," "+prefix_words+row.ɪpas+" ")
  all_sentences_arr=all_sentences.split(prefix_sentence);
  for i,row in enumerate(subset.itertuples()):
    
    validated_ipa=row.validated_ipa.split(" ")
    ith_sentence=all_sentences_arr[i+1].split(" ")

    for i,word in enumerate(ith_sentence):
      if word.startswith(prefix_words):
        validated_ipa[i+1]=word
    validated_ipa=" ".join(validated_ipa)
    validated_ipa=validated_ipa.replace(prefix_words,"")
    df.at[row.Index,"validated_ipa"]=validated_ipa
      
    # df.at[row.Index,"temp_sentence"]=all_sentences_arr[row.Index+1]

    

In [67]:
# threadpool make sure everything is currect before running this cell
start=0
steps=10
end=100
subsets=[]
start_pos=start
while start<end:
  subsets.append(df.iloc[start:start+steps])
  start=start+steps
pool=ThreadPoolExecutor(len(subsets))
ws=[]
for subset in subsets:
  ws.append(pool.submit(replace_words, subset))
print(pool.shutdown())


None


In [68]:
df.to_csv(f"ebanglalibrary_100k_with_ipa0_106500_validated_{start_pos}_{end}.csv",index=False)

In [23]:
name="sourav ahmed"
# name[-1]="x"
name=name[:-1] + "x"
print(name.replace("a","A"))
print(name)

sourAv Ahmex
sourav ahmex


In [8]:
unique_char_bangla=[' ', '!', '#', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '~', '\xa0', '£', 'ª', '\xad', '¯', '·', 'Æ', 'â', '˃', 'ا', 'ث', 'خ', 'د', 'ر', 'س', 'ص', 'ع', 'ق', 'ل', 'ه', 'ي', '।', '॥', 'ঁ', 'ং', 'ঃ', 'অ', 'আ', 'ই', 'ঈ', 'উ', 'ঊ', 'ঋ', 'এ', 'ঐ', 'ও', 'ঔ', 'ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ', 'জ', 'ঝ', 'ঞ', 'ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ত', 'থ', 'দ', 'ধ', 'ন', 'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', 'ল', 'শ', 'ষ', 'স', 'হ', '়', 'া', 'ি', 'ী', 'ু', 'ূ', 'ৃ', 'ে', 'ৈ', 'ো', 'ৌ', '্', 'ৎ', 'ৗ', 'ড়', 'ঢ়', 'য়', '০', '১', '২', '৩', '৪', '৫', '৬', '৭', '৮', '৯', 'ৰ', 'ৱ', '৷', '৺', 'ਆ', '\u200c', '\u200d', '‒', '–', '—', '‘', '’', '“', '”', '•', '…', '−', '√', '∣', '⋯', '\uf020', '\uf02c', '\uf02d', '\uf05b', '\ufeff']
unique_char_ipa=[' ', '!', '%', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '~', '\xa0', '£', '\xad', 'æ', 'ŋ', 'ɔ', 'ɟ', 'ɽ', 'ʃ', 'ʰ', 'ʱ', 'ʲ', 'ʷ', '̃', '̪', '̯', '͡', 'ا', 'ث', 'خ', 'د', 'ر', 'س', 'ص', 'ع', 'ق', 'ل', 'ه', 'ي', '।', '॥', '৷', '–', '—', '‘', '’', '“', '”', '•', '…', '∣', '\ufeff']

# for row in df.itertuples():
#   for ch in row.ipa:
#     if ch not in unique_char_ipa:
#       unique_char_ipa.append(ch)
# unique_char_ipa.sort()
# print(unique_char_ipa)

MISC


In [ ]:
      # if ch=='ʲ' and (row.sentence[i+1]==" " or row.sentence[i+1]=="\n" or  row.sentence[i+1]=="।" or row.sentence[i+1]=="?" or row.sentence[i+1]=="!" or row.sentence[i+1]=="%" or ):
